In [1]:
import pandas as pd
import numpy as np

import random
from scipy.stats import truncnorm
import uuid

# Requisito 1

In [3]:
d1 = {
    "ds": "dataset1",
    "columns": [
        {
            "name": "area",
            "type": "category",
            "values": ["TI", "FIN", "HR"]
        },
        {
            "name": "id",
            "type": "unique"
        }
    ],
    "random": False
}

In [4]:
# Función encargada de calcular la columna con mayor cantidad de registros
def calc_greater(cols_arr):
    max_values = 0
    
    for col in cols_arr:
        col_type = col['type']

        if col_type == 'foreign':
            continue  # Ignorar, ya que no tiene valores directos

        if col_type == 'date':
            max_values = max(max_values, pd.date_range(start=col['values']['min'], end=col['values']['max']).size)

        if col_type == 'category':
            max_values = max(max_values, len(col['values']))

    return max_values

In [5]:
def generate_unique_id(n):
    return [str(uuid.uuid4()).replace('-', '')[:16] for _ in range(n)]

# Determinar el número máximo de filas basado en la columna con más datos
max_rows = calc_greater(d1["columns"])

# Crear un diccionario para almacenar los datos del DataFrame
data = {}

# Generar los datos para cada columna en la configuración
for column in d1["columns"]:
    column_name = column["name"]
    column_type = column["type"]
    
    if column_type == "category":
        values = column["values"]
        # Repetir los valores para llenar hasta el número máximo de filas
        repeated_values = (values * (max_rows // len(values) + 1))[:max_rows]
        data[column_name] = pd.Categorical(repeated_values)
    elif column_type == "unique":
        data[column_name] = generate_unique_id(max_rows)
    else:
        raise ValueError(f"Tipo de columna no soportado: {column_type}")


# Crear el DataFrame
df = pd.DataFrame(data)

# Imprimir el DataFrame
df

,area,id
0,TI,ee5a309492584d67
1,FIN,f099ebf84cdd42a0
2,HR,9b101c6762714cac


# Requisito 2

In [7]:
d2 = {
    "ds": "dataset2",
    "columns": [
        {
            "name": "id",
            "type": "unique"
        },
        {
            "name": "area",
            "type": "foreign",
            "values": "dataset1.area"
        },
        {
            "name": "subarea",
            "type": "category",
            "values": ["SA1", "SA2", "SA3", "SA4"]
        }
    ],
    "random": False
}

In [8]:
# Verificacion
referenced_dataset = d1  # Referencia al dataset 1

# Verificar que el dataset de referencia exista
if d2["columns"][1]["values"].split('.')[0] != referenced_dataset['ds']:
    raise ValueError(f"Referencia inválida al dataset. Dataset válido: {referenced_dataset['ds']}")

# Verificar que la columna de referencia exista
if d2["columns"][1]["values"].split('.')[1] not in [col['name'] for col in referenced_dataset['columns']]:
    raise ValueError(f"Columna de referencia inválida. Columnas válidas: {[col['name'] for col in referenced_dataset['columns']]}")

In [9]:
# Determinar el número máximo de filas basado en la columna con más datos
max_rows = calc_greater(d2["columns"])

In [10]:
data2 = {}

# Selección aleatoria de valores para la columna 'area' del dataset 1
random_areas = [random.choice(referenced_dataset["columns"][0]["values"]) for _ in range(max_rows)]

# Generar los datos para cada columna en la configuración del dataset2
for column in d2["columns"]:
    column_name = column["name"]
    column_type = column["type"]
    
    if column_type == "category":
        values = column["values"]
        # Repetir los valores para llenar hasta el número máximo de filas
        repeated_values = (values * (max_rows // len(values) + 1))[:max_rows]
        data2[column_name] = pd.Categorical(repeated_values)
    elif column_type == "unique":
        data2[column_name] = generate_unique_id(max_rows)
    elif column_type == "foreign":
        # Repetir los valores del dataset1.area para llenar hasta el número máximo de filas
        repeated_values = (random_areas * (max_rows // len(random_areas) + 1))[:max_rows]
        data2[column_name] = pd.Categorical(repeated_values)
    else:
        raise ValueError(f"Tipo de columna no soportado: {column_type}")

# Crear el DataFrame del dataset2
df2 = pd.DataFrame(data2)

# Imprimir el DataFrame del dataset2
df2

,id,area,subarea
0,4b7f8c716ffb403b,FIN,SA1
1,0f1d8637a7c54921,TI,SA2
2,d3ed2db52bb14552,HR,SA3
3,f88a87ba3c2c44ab,FIN,SA4


# Requisito 3

In [12]:
d3 = {
    "ds": "dataset3",
    "columns": [
        {
            "name": "id",
            "type": "unique"
        },
        {
            "name": "subarea",
            "type": "foreign",
            "values": "dataset2.id"
        },
        {
            "name": "income",
            "type": "numeric",
            "values": {
                "min": 10000,
                "max": 100000
            }
        },
        {
            "name": "goal",
            "type": "numeric",
            "values": {
                "min": 10000,
                "max": 100000,
                "std": 50000,
                "mean": 50000
            }
        }
    ],
    "random": True,
    "random_rows": 1000
}

In [13]:
# Función para generar valores utilizando una distribución normal
def generate_normal(min_val, max_val, size):
    mean = (min_val + max_val) / 2
    std = (max_val - min_val) / 4  # Aproximación: usar 1/4 del rango como desviación estándar
    return np.random.normal(loc=mean, scale=std, size=size).astype(int).clip(min_val, max_val)

# Función para generar valores utilizando una distribución normal truncada
def generate_truncated_normal(mean, std, min_val, max_val, size):
    a, b = (min_val - mean) / std, (max_val - mean) / std
    return truncnorm.rvs(a, b, loc=mean, scale=std, size=size).astype(int)

# Generar el número de filas basado en random_rows
num_rows = d3["random_rows"]

# Crear un diccionario para almacenar los datos del dataset3
data3 = {}

In [14]:
# Generar valores para cada columna en la configuración de d3
for column in d3["columns"]:
    column_name = column["name"]
    column_type = column["type"]

    if column_type == "unique":
        data3[column_name] = generate_unique_id(num_rows)

    elif column_type == "foreign":
        if column_name == "subarea":
            #subarea_ids = [str(uuid.uuid4()).replace('-', '')[:16] for _ in range(max_rows)]
            #data3[column_name] = [random.choice(subarea_ids) for _ in range(num_rows)]
            data3[column_name] = [random.choice(df2["id"]) for _ in range(num_rows)]
        else:
            values = column["values"]
            # Repetir los valores para llenar hasta el número máximo de filas
            repeated_values = (values * (num_rows // len(values) + 1))[:num_rows]
            data3[column_name] = pd.Categorical(repeated_values)

    elif column_type == "numeric":
        values_config = column["values"]
        min_val = values_config["min"]
        max_val = values_config["max"]

        if "mean" in values_config and "std" in values_config:
            mean = values_config["mean"]
            std = values_config["std"]
            data3[column_name] = generate_truncated_normal(mean, std, min_val, max_val, num_rows)
        else:
            data3[column_name] = generate_normal(min_val, max_val, num_rows)

# Crear el DataFrame del dataset3
df3 = pd.DataFrame(data3)

# Imprimir el DataFrame del dataset3
df3

,id,subarea,income,goal
0,1d788455e6e54cf9,0f1d8637a7c54921,41507,24801
1,5ec92cd4dcfa404b,d3ed2db52bb14552,51437,53817
2,04c2b6adaa3c43ba,f88a87ba3c2c44ab,59666,85844
3,66f4542a59e14cc8,d3ed2db52bb14552,41568,67625
4,d65198657691408a,f88a87ba3c2c44ab,62729,78634
...,...,...,...,...
995,01d7a325145e4b4d,f88a87ba3c2c44ab,43976,21602
996,f5f0dc0044144b7b,4b7f8c716ffb403b,32542,40909
997,38aa7ba679d94c0d,4b7f8c716ffb403b,54618,70320
998,2b908a4c4415448d,0f1d8637a7c54921,48580,33184


# Requisito 4

In [16]:
# Configuración de d4 y d5
d4 = {
    "ds": "dataset4",
    "columns": [
        {
            "name": "id",
            "type": "unique"
        },
        {
            "name": "department",
            "type": "category",
            "values": ["Sales", "Engineering", "Marketing", "Finance", "HR"]
        },
        {
            "name": "age",
            "type": "numeric",
            "values": {
                "min": 22,
                "max": 65
            }
        },
        {
            "name": "experience",
            "type": "numeric",
            "values": {
                "min": 1,
                "max": 40,
                "mean": 15,
                "std": 10
            }
        }
    ],
    "random": False
}

d5 = {
    "ds": "dataset5",
    "columns": [
        {
            "name": "id",
            "type": "unique"
        },
        {
            "name": "employee_id",
            "type": "foreign",
            "values": "dataset4.id"
        },
        {
            "name": "country",
            "type": "category",
            "values": ["USA", "Canada", "UK", "Germany", "France", "India", "China", "Japan"]
        },
        {
            "name": "status",
            "type": "category",
            "values": ["Full-time", "Part-time", "Contract", "Intern"]
        },
        {
            "name": "salary",
            "type": "numeric",
            "values": {
                "min": 30000,
                "max": 150000
            }
        },
        {
            "name": "bonus",
            "type": "numeric",
            "values": {
                "min": 10000,
                "max": 20000,
                "mean": 50000,
                "std": 30000
            }
        },
        {
            "name": "joining_date",
            "type": "date",
            "values": {
                "min": "2000-01-01",
                "max": "2023-12-31"
            }
        }
    ],
    "random": True,
    "random_rows": 1000
}


In [17]:
# Función para generar fechas aleatorias
def get_random_dates(min_date, max_date, n):
    items = pd.date_range(start=min_date, end=max_date, freq='D')
    return random.choices(items, k=n)

In [18]:
# Generar valores de categoría aleatorios
def generate_category_random_values(values, size):
    return np.random.choice(values, size=size, replace=True)

In [19]:
def generate_dataset(config, reference_data=None):
    num_rows = config["random_rows"] if config["random"] else calc_greater(config["columns"])
    data = {}

    for column in config["columns"]:
        column_name = column["name"]
        column_type = column["type"]

        if column_type == "unique":
            data[column_name] = generate_unique_id(num_rows)

        elif column_type == "category":
            values = column["values"]
            if isinstance(values, list):
                repeated_values = (values * (num_rows // len(values) + 1))[:num_rows]
                data[column_name] = pd.Categorical(repeated_values)
            elif isinstance(values, str) and reference_data and values in reference_data:
                ref_values = reference_data[values]
                repeated_values = (ref_values * (num_rows // len(ref_values) + 1))[:num_rows]
                data[column_name] = pd.Categorical(repeated_values)

        elif column_type == "numeric":
            values_config = column["values"]
            min_val = values_config["min"]
            max_val = values_config["max"]

            if "mean" in values_config and "std" in values_config:
                mean = values_config["mean"]
                std = values_config["std"]
                data[column_name] = generate_truncated_normal(mean, std, min_val, max_val, num_rows)
            else:
                data[column_name] = generate_normal(min_val, max_val, num_rows)

        elif column_type == "date":
            start_date = pd.to_datetime(column["values"]["min"])
            end_date = pd.to_datetime(column["values"]["max"])
            data[column_name] = get_random_dates(start_date, end_date, num_rows)

        elif column_type == "foreign":
            if reference_data is not None and column["values"] in reference_data:
                values = reference_data[column["values"]]
                repeated_values = (values * (num_rows // len(values) + 1))[:num_rows]
                data[column_name] = pd.Categorical(repeated_values)

    return pd.DataFrame(data)



# Generar el dataset d4
df4 = generate_dataset(d4)

# Generar el dataset d5 usando los datos del dataset d4
reference_data = {"dataset4.id": df4["id"].tolist()}
df5 = generate_dataset(d5, reference_data)

In [20]:
print("Dataset4:")
df4

Dataset4:


,id,department,age,experience
0,a62a2228c081483c,Sales,65,22
1,4c356cd4e86c4730,Engineering,48,17
2,581370d3c5954085,Marketing,43,16
3,591fbbcef31f4fae,Finance,37,19
4,b4262858f23346b3,HR,51,13


In [21]:
print("\nDataset5:")
df5


Dataset5:


,id,employee_id,country,status,salary,bonus,joining_date
0,bb3b1186a0354a13,a62a2228c081483c,USA,Full-time,103721,13260,2005-08-05
1,d86d29d2a99c4deb,4c356cd4e86c4730,Canada,Part-time,51836,14619,2014-12-01
2,d21c9752fe2d475b,581370d3c5954085,UK,Contract,55183,18987,2007-01-10
3,5a8155643d6c425d,591fbbcef31f4fae,Germany,Intern,72046,14483,2005-03-01
4,acf5b59f925d4724,b4262858f23346b3,France,Full-time,70883,18226,2017-06-15
...,...,...,...,...,...,...,...
995,b41550d8cb0944f9,a62a2228c081483c,Germany,Intern,39771,18299,2002-05-01
996,6bb9e898a8684b0a,4c356cd4e86c4730,France,Full-time,113047,17894,2015-12-03
997,d33f40f827d74a5c,581370d3c5954085,India,Part-time,102329,13475,2008-04-18
998,cbe525d2e5404496,591fbbcef31f4fae,China,Contract,130502,14924,2017-08-05


In [22]:
# Generar valores de clave externa aleatorios
def generate_foreign_random_values(reference_values, size):
    return np.random.choice(reference_values, size=size, replace=True)

# Extraer dependencias de un dataset
def extract_dependencies(obj):
    dependencies = []
    for col in obj['columns']:
        if col['type'] == 'foreign':
            df_name, _ = col['values'].split('.')
            dependencies.append(df_name)
    return dependencies

def reorder_by_dependencies(configs):
    sorted_configs = []
    remaining_configs = configs.copy()
    
    while remaining_configs:
        for config in remaining_configs:
            dependencies = extract_dependencies(config)
            if all(dep in [c['ds'] for c in sorted_configs] for dep in dependencies):
                sorted_configs.append(config)
                remaining_configs.remove(config)
                
    return sorted_configs

all_dfs = {}

def build_dataframes(conf_list):
    # Reordenar las configuraciones por dependencias
    sorted_conf_list = reorder_by_dependencies(conf_list)
    
    # Crear los DataFrames en el orden correcto
    for config in sorted_conf_list:
        reference_data = {f"{k}.{col}": v[col].tolist() for k, v in all_dfs.items() for col in v.columns}
        df = generate_dataset(config, reference_data)
        df.name = config['ds']
        all_dfs[df.name] = df
        
    # Reordenar según el orden original de conf_list
    return [all_dfs[config['ds']] for config in conf_list]



# Definir la lista de configuraciones
conf_list = [d2, d1, d5, d3, d4]

# Generar los DataFrames en el orden correcto
dataframe_list = build_dataframes(conf_list)

# Imprimir los DataFrames generados
#for df in dataframe_list:
 #   print(f"DataFrame {df.name}:\n", df.head(), "\n")

In [23]:
dataframe_list[2]

,id,employee_id,country,status,salary,bonus,joining_date
0,4ba5c75e35c645e1,e48b11ddd31d4274,USA,Full-time,126089,17566,2014-09-26
1,6104cf1c7fd54a99,540f939698d4425e,Canada,Part-time,61842,18432,2005-09-30
2,ff34b5b4b10e4445,d22f7c1040064a98,UK,Contract,102003,13653,2023-09-17
3,48e1172366fa4db0,5250b1c474614855,Germany,Intern,49663,15637,2016-04-28
4,52fa417684e94466,d72bcc6a0d8c453e,France,Full-time,79374,19643,2018-06-09
...,...,...,...,...,...,...,...
995,d7ef4dc820094e50,e48b11ddd31d4274,Germany,Intern,55907,16172,2010-02-11
996,2034fc9837eb4739,540f939698d4425e,France,Full-time,127393,17147,2000-05-12
997,3477d26750534736,d22f7c1040064a98,India,Part-time,113223,13591,2010-01-19
998,208a9bdda7554aa6,5250b1c474614855,China,Contract,114418,12901,2012-08-16


# Requisito 5

In [25]:
simulation_extended = dataframe_list[2]

In [26]:
simulation_extended

,id,employee_id,country,status,salary,bonus,joining_date
0,4ba5c75e35c645e1,e48b11ddd31d4274,USA,Full-time,126089,17566,2014-09-26
1,6104cf1c7fd54a99,540f939698d4425e,Canada,Part-time,61842,18432,2005-09-30
2,ff34b5b4b10e4445,d22f7c1040064a98,UK,Contract,102003,13653,2023-09-17
3,48e1172366fa4db0,5250b1c474614855,Germany,Intern,49663,15637,2016-04-28
4,52fa417684e94466,d72bcc6a0d8c453e,France,Full-time,79374,19643,2018-06-09
...,...,...,...,...,...,...,...
995,d7ef4dc820094e50,e48b11ddd31d4274,Germany,Intern,55907,16172,2010-02-11
996,2034fc9837eb4739,540f939698d4425e,France,Full-time,127393,17147,2000-05-12
997,3477d26750534736,d22f7c1040064a98,India,Part-time,113223,13591,2010-01-19
998,208a9bdda7554aa6,5250b1c474614855,China,Contract,114418,12901,2012-08-16


In [27]:
category_cols = ['country', 'status']
numeric_cols = ['salary', 'bonus']

size = 100000

In [28]:
def get_categorical_dataset_simulated(simulation_extended, category_cols, size):
      # Verifica la existencia de las columnas categóricas
    for col in category_cols:
        if col not in simulation_extended.columns:
            raise ValueError(f"Column {col} not found in DataFrame")

    # La columna n = 1 servirá para realizar la función de agrupación count en el DataFrame
    simulation_extended['n'] = 1

    # Generamos la agrupación por columnas categóricas con su respectiva función de agregación count
    grouped_combination = simulation_extended.groupby(
        category_cols,
        as_index=False,
        observed=True  # Cambiado a True para incluir solo combinaciones presentes
    ).agg({
        'n': 'count'
    })

    # Renombramos las columnas
    grouped_combination.columns = category_cols + ['count']

    # Calculamos la probabilidad de cada combinación
    total_rows = simulation_extended.shape[0]
    if total_rows <= 0:
        raise ValueError("Total rows in simulation_extended is zero or negative")
    
    grouped_combination['prob'] = grouped_combination['count'] / total_rows

    # Verificamos la suma de las probabilidades y normalizamos si es necesario
    prob_sum = grouped_combination['prob'].sum()
    if not np.isclose(prob_sum, 1):
        grouped_combination['prob'] /= prob_sum

    # Generamos aleatoriamente un arreglo de índices
    rand_index = np.random.choice(
        grouped_combination.index,
        size=size,
        p=grouped_combination['prob']
    )

    # Retornamos los 100k registros aleatorios ya con sus respectivos valores
    return grouped_combination.loc[rand_index, category_cols]


# Generar los registros de manera aleatoria para las columnas numericas
def get_numeric_column_simulated( simulated , simulation_extended , category_cols , column_name ):
    # Funcion de agregacion para obtener el min, max, mean y std por cada agrupacion de columnas categoricas
    a1 = simulation_extended.groupby(
        category_cols
        , as_index = False,
        observed=True
    ).agg(
        {
            column_name: ["min","max","mean","std"]
        }
    )

    a1.columns = [c for c in category_cols] + ["Min", "Max", "Mean", "Std"]  # Renombra las columnas de a1

    ColumnSimulated = pd.DataFrame() # Inicializa un nuevo dataframe vacio en este caso
    
    # Recorre los indices de a1
    for i in a1.index:
        # Toma el registro completo del indice i, el registro completo son toda la fila de valores de ese indice
        rs = a1.loc[i]

        # reasigna final_simulation y crea una copia de todos los registros de simulated (que al principio era una copia de final_simulation sin la columna n) que contengan el indice i
        final_simulation = simulated.loc[i].copy()    

        # Genera los valores aleatorios en base a su resumen estadistico de min, max, mean y std
        data = generate_truncated_normal(
            rs["Min"] - 1 if rs["Std"] == 0 else rs["Min"],
            rs["Max"] + 1 if rs["Std"] == 0 else rs["Max"],
            1 if rs["Std"] == 0 else rs["Std"],
            rs["Mean"], 
            final_simulation.shape[0]
        )

        # Agregamos la nueva columna a final_simulation
        final_simulation[column_name] = data

        # Concatenamos el conjunto de registros anteriores con el registro actual recorrido del ciclo
        ColumnSimulated = pd.concat( [ ColumnSimulated , final_simulation  ] )

    return ColumnSimulated.reset_index(drop=True) # Retornamos el resultado final restableciendo los indices

# Analisis mejor combinacion de variables categoricas

In [31]:
simulation_extended['n'] = 1
grouped_combination = simulation_extended.groupby(
    category_cols,
    as_index=False,
    observed=True
).agg({
    'n': 'count'
})

grouped_combination.columns =  category_cols + ['count']
grouped_combination['prob'] = grouped_combination['count'] / dataframe_list[4].shape[0]

In [32]:
final_simulation = get_categorical_dataset_simulated(simulation_extended, category_cols, 100000)
final_simulation

,country,status
1,China,Contract
4,India,Part-time
0,Canada,Part-time
6,UK,Contract
6,UK,Contract
...,...,...
7,USA,Full-time
0,Canada,Part-time
1,China,Contract
7,USA,Full-time


# Analisis mejor combinacion de variables categoricas despues de generar los n cantidad de registros

In [33]:
final_simulation['n'] = 1
result = final_simulation.groupby(
    category_cols,
    as_index=False,
    observed=True
).agg({
    'n': 'count'
})

result.columns = category_cols + ['count']
result['prob'] = result['count'] / final_simulation.shape[0]

In [34]:
# Mejor combinacion de variables categoricas desde la primera agrupacion
grouped_combination.sort_values(by='count', ascending=False)

,country,status,count,prob
0,Canada,Part-time,125,25.0
1,China,Contract,125,25.0
2,France,Full-time,125,25.0
3,Germany,Intern,125,25.0
4,India,Part-time,125,25.0
5,Japan,Intern,125,25.0
6,UK,Contract,125,25.0
7,USA,Full-time,125,25.0


In [35]:
# Probabilidades de las mejores combinaciones despues de haber generado los 100k registros
result.sort_values(by='count', ascending=False)

,country,status,count,prob
1,China,Contract,12640,0.12640
6,UK,Contract,12611,0.12611
7,USA,Full-time,12568,0.12568
2,France,Full-time,12553,0.12553
3,Germany,Intern,12453,0.12453
5,Japan,Intern,12439,0.12439
0,Canada,Part-time,12388,0.12388
4,India,Part-time,12348,0.12348


In [36]:
final_simulation

,country,status,n
1,China,Contract,1
4,India,Part-time,1
0,Canada,Part-time,1
6,UK,Contract,1
6,UK,Contract,1
...,...,...,...
7,USA,Full-time,1
0,Canada,Part-time,1
1,China,Contract,1
7,USA,Full-time,1


In [37]:
# simulated en este caso es una copia de final_simulation excluyendo la variable n que no pertenece al dataframe original
# final_simulation contiene los 100k registros generados aleatoriamente
simulated = final_simulation.drop(columns='n', axis=1).copy()

# Restablece los indices a una secuencia por default antes: [0, 0, 0, ..., n, n] ahora: [0,1,2,3,...,n]
final_simulation = simulated.sort_index().reset_index(drop=True).copy()

# Recorremos el arreglo de columnas numericas
for nc in numeric_cols:
    # simulated = copia de los 100k registros generados aleatorios
    # simulation_extended = dataframe al cual se le esta realizando el analisis. En este caso este dataframe se creo desde el inicio con 1k registros
    # nc = columna numerica actual a medida se recorre el ciclo de las columnas numericas
    dfn = get_numeric_column_simulated(simulated, simulation_extended, category_cols, nc) 

    # Combina el dataframe actual de final_simulation con el dataframe que se va generando a medida recorra el ciclo
    # Los combina por sus indices (Tomando en cuenta que los indices son unicos)
    final_simulation = pd.merge(
        final_simulation
        , dfn.loc[:,[nc]]
        , left_index=True
        , right_index=True
    )

In [38]:
final_simulation

,country,status,salary,bonus
0,Canada,Part-time,55121,3227
1,Canada,Part-time,69810,4363
2,Canada,Part-time,70030,3088
3,Canada,Part-time,36996,6130
4,Canada,Part-time,35976,7697
...,...,...,...,...
99995,USA,Full-time,68597,6224
99996,USA,Full-time,48096,5672
99997,USA,Full-time,80757,4151
99998,USA,Full-time,40767,6941
